# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Vietnamnet - Bài tập)**


> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```


Cũng giống như VNExpress - Demo

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
     {
        "url": "https://vietnamnet.vn/khi-gap-van-xui-hay-nhin-3-...",
        "title": "Khi gặp vận xui, hãy nhìn 3 bức ảnh này sẽ thấy cuộc đời ...",
        "description": "Cuộc đời như thế nào tùy thuộc vào góc nhìn của từng ...",
        "content": "Bức tranh 1: Có 3 hay 4 khúc gỗ?\nCâu trả lời là 3 hoặc 4 khúc gỗ ...",
        "metadata": {
            "cat": "ĐỜI SỐNG",
            "subcat": "GIỚI TRẺ",
            "published_date": 1721274540,
            "author": "Phan Hằng"
        },
    },
    ```


# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [2]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
# !apt install -qq chromium-chromedriver

In [3]:
# Install selenium
%pip install -qq selenium

# Tạo thư mục để chứa data
# !mkdir -p data

Note: you may need to restart the kernel to use updated packages.


In [4]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [5]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [6]:
MAGAZINE_NAME = "vietnamnet"
HOME_PAGE = "https://vietnamnet.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [7]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [8]:
# Chọn and click menu buttons

#### CODE
menu_show_all = driver.find_element(by=By.CSS_SELECTOR, value=".icon-hamburger")
menu_show_all.click()
####

In [9]:
# Lấy hết tất cả thể loại từ đây
cats = []

# bỏ qua logo-premium, logo-htvn

#### CODE
ul_cats = driver.find_element(by=By.CLASS_NAME, value="header_submenu-content-list")
li_cats = ul_cats.find_elements(by=By.TAG_NAME, value="li")
for li in li_cats:
    if li.get_attribute("class").startswith("logo"): continue
    a = li.find_element(by=By.TAG_NAME, value="a")
    cats.append({
        "cat_name": a.get_attribute("title").strip(),
        "url": a.get_attribute("href").strip()    
    })
####

In [10]:
len(cats), cats

(28,
 [{'cat_name': 'Chính trị', 'url': 'https://vietnamnet.vn/thoi-su/chinh-tri'},
  {'cat_name': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_name': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_name': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_name': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_name': 'Giải trí', 'url': 'https://vietnamnet.vn/giai-tri'},
  {'cat_name': 'Văn hóa', 'url': 'https://vietnamnet.vn/van-hoa'},
  {'cat_name': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_name': 'Thông tin và Truyền thông',
   'url': 'https://vietnamnet.vn/thong-tin-truyen-thong'},
  {'cat_name': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_name': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_name': 'Bất động sản', 'url': 'https://

In [11]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [12]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 10 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vietnamnet_url.json"

EXCLUDING_CATEGORIES = ["Giảm nghèo bền vững", "English", "Toàn văn"] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

#### Chạy thử nghiệm



In [13]:
# Vào thử một url
driver.get(cats[0]['url'])

In [14]:
# Get all titles item

#### CODE
news_items = driver.find_elements(by=By.CLASS_NAME, value="version-news")
news_items = [news.find_element(by=By.CLASS_NAME, value="vnn-title").find_element(by=By.TAG_NAME, value="a") for news in news_items]
for news in news_items:
    print(news.get_attribute("title"), news.get_attribute("href"))
####

Tổng Bí thư Tô Lâm: Sớm kiện toàn các chức danh lãnh đạo để chuẩn bị Đại hội 14 của Đảng https://vietnamnet.vn/tong-bi-thu-to-lam-som-kien-toan-cac-chuc-danh-lanh-dao-chuan-bi-dai-hoi-dang-2309262.html
Thường vụ Quốc hội sẽ chất vấn nhiều Bộ trưởng về 2 nhóm lĩnh vực https://vietnamnet.vn/thuong-vu-quoc-hoi-se-chat-van-nhieu-bo-truong-ve-2-nhom-linh-vuc-2309295.html
Đề nghị Bộ Quốc phòng Nhật Bản hợp tác chuyển giao công nghệ, an ninh mạng https://vietnamnet.vn/de-nghi-bo-quoc-phong-nhat-ban-hop-tac-chuyen-giao-cong-nghe-an-ninh-mang-2309288.html
Thủ tướng: Kiện toàn cơ cấu tổ chức của Chính phủ theo hướng tinh gọn https://vietnamnet.vn/thu-tuong-kien-toan-co-cau-to-chuc-cua-chinh-phu-theo-huong-tinh-gon-2309284.html
Lãnh đạo các nước muốn hợp tác chặt chẽ với Tổng Bí thư, Chủ tịch nước Tô Lâm https://vietnamnet.vn/lanh-dao-cac-nuoc-muon-hop-tac-chat-che-voi-tong-bi-thu-chu-tich-nuoc-to-lam-2309280.html
Tổng Bí thư, Chủ tịch nước Tô Lâm chủ trì cuộc họp lãnh đạo chủ chốt https://vietna

In [15]:
# Tìm next page

#### CODE
for _ in range(10):
    next_page_elem = driver.find_element(by=By.CLASS_NAME, value="pagination-next")
    link = next_page_elem.find_element(By.TAG_NAME, "a").get_attribute("href")
    print(link)
    next_page_elem.click()
####

https://vietnamnet.vn/chinh-tri-page1
https://vietnamnet.vn/chinh-tri-page2
https://vietnamnet.vn/chinh-tri-page3
https://vietnamnet.vn/chinh-tri-page4
https://vietnamnet.vn/chinh-tri-page5
https://vietnamnet.vn/chinh-tri-page6
https://vietnamnet.vn/chinh-tri-page7
https://vietnamnet.vn/chinh-tri-page8
https://vietnamnet.vn/chinh-tri-page9
https://vietnamnet.vn/chinh-tri-page10


In [16]:
driver.close()

#### Chạy thật

In [17]:
NUM_ARTICLES_PER_CAT = 400
driver = webdriver.Chrome(options=chrome_options)

In [18]:
# Global variables for filtering deduplicating urls
global_urls = set()

def crawl_each_category_url(driver: webdriver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    # Get all item-news class item
    
    #### CODE HERE
    cat_urls = set()
    page_url = category_url
    while len(cat_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(page_url)
        news_items = driver.find_elements(by=By.CLASS_NAME, value="version-news")
        for news in news_items:
            try:
                title = news.find_element(By.CLASS_NAME, "vnn-title").find_element(By.TAG_NAME, "a")
                url = title.get_attribute("href").strip()
                if url.startswith(HOME_PAGE) and url not in global_urls:
                    cat_urls.add(url)
                    global_urls.add(url)  
                # print(news.get_attribute("title"), news.get_attribute("href"))
            except StaleElementReferenceException:
                print("Check driver reference, consider re-run notebook")
                continue
            except NoSuchElementException:
                print("Check code logic and edge cases !! (No Such Element)")
                continue
        page_url = driver.find_element(By.CLASS_NAME, "pagination-next").find_element(By.TAG_NAME, "a").get_attribute("href")
    print(len(cat_urls))
    ####
    return cat_urls

In [19]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat["cat_name"]
    url = cat['url']
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

You are at {'cat_name': 'Chính trị', 'url': 'https://vietnamnet.vn/thoi-su/chinh-tri'}.
406
You are at {'cat_name': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'}.
419
You are at {'cat_name': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'}.
402
You are at {'cat_name': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'}.
406
You are at {'cat_name': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'}.
405
You are at {'cat_name': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'}.
404
You are at {'cat_name': 'Giải trí', 'url': 'https://vietnamnet.vn/giai-tri'}.
404
You are at {'cat_name': 'Văn hóa', 'url': 'https://vietnamnet.vn/van-hoa'}.
420
You are at {'cat_name': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'}.
405
You are at {'cat_name': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'}.
405
You are at {'cat_name': 'Thông tin và Truyền thông', 'url': 'https://vietnamnet.vn/thong-tin-truyen-thong'}.
400
You are at {'cat_name': 'Pháp luật', 'url': 'https://viet

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagination-next"}
  (Session info: chrome-headless-shell=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C5699632+30946]
	(No symbol) [0x00007FF6C564E3C9]
	(No symbol) [0x00007FF6C5546FDA]
	(No symbol) [0x00007FF6C559822C]
	(No symbol) [0x00007FF6C559850C]
	(No symbol) [0x00007FF6C55DDCB7]
	(No symbol) [0x00007FF6C55BCAAF]
	(No symbol) [0x00007FF6C55DB041]
	(No symbol) [0x00007FF6C55BC813]
	(No symbol) [0x00007FF6C558A6E5]
	(No symbol) [0x00007FF6C558B021]
	GetHandleVerifier [0x00007FF6C57CF83D+1301229]
	GetHandleVerifier [0x00007FF6C57DBDB7+1351783]
	GetHandleVerifier [0x00007FF6C57D2A03+1313971]
	GetHandleVerifier [0x00007FF6C56CDD06+245686]
	(No symbol) [0x00007FF6C565758F]
	(No symbol) [0x00007FF6C5653804]
	(No symbol

In [20]:
with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

In [21]:
len(global_urls)

9766

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [22]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vietnamnet_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 400 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/vietnamnet"
!mkdir "$DATA_FOLDER_OUTPUT"

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

A subdirectory or file data/vietnamnet already exists.


In [23]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn:
    url_data = json.load(fIn)

len(url_data)

24

#### Chạy thử nghiệm

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
    "https://vietnamnet.vn/bat-giam-nguyen-thu-truong-bo-tai-nguyen-va-moi-truong-nguyen-linh-ngoc-2304574.html",
    "https://vietnamnet.vn/ky-su-cong-nghe-ve-que-nuoi-oc-thu-nhe-lai-nua-ty-moi-nam-2304196.html",
    "https://vietnamnet.vn/nsnd-thanh-hoa-va-nsnd-le-dung-hai-nguoi-thay-dac-biet-cua-nsut-viet-hoan-2304214.html",
    #author in last
    "https://vietnamnet.vn/doi-giay-cu-mon-giua-gia-lanh-chau-au-cua-tong-bi-thu-nguyen-phu-trong-2303988.html",
    #class maincontent, table ignore
    "https://vietnamnet.vn/lo-dien-cao-toc-dep-bac-nhat-dat-nuoc-chay-xuyen-rung-ngap-man-2303643.html",
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[-1]

In [ ]:
driver.get(SAMPLE_ARTICLE_URL)

In [ ]:
# Tìm kiếm title

#### CODE HERE
driver.find_element(By.CLASS_NAME, "content-detail-title").text
####


'Lộ diện cao tốc đẹp bậc nhất đất nước chạy xuyên rừng ngập mặn'

In [ ]:
# Tìm kiếm description

#### CODE HERE
driver.find_element(By.CLASS_NAME, "content-detail-sapo").text

####


'Dự án cao tốc Bến Lức - Long Thành đi qua Đồng Nai dài 27km. Trong đó, đoạn băng qua rừng ngập mặn dài 3,5km đang gấp rút thi công và dự kiến hoàn thành trong tháng 7 này.'

In [ ]:
# Thu thập thể loại

#### CODE HERE
driver.find_element(By.CSS_SELECTOR, ".bread-crumb-detail > ul > li > a[title]").get_attribute("title")
####

'Thời sự'

In [ ]:
#Thu thập ngày
#### CODE HERE
driver.find_element(By.CLASS_NAME, "bread-crumb-detail__time").text
####


'chủ nhật, 21/07/2024 - 06:00'

In [ ]:
# Tìm kiếm contents
#### CODE HERE
article = driver.find_element(By.CLASS_NAME, "main-content")
children = article.find_elements(By.XPATH, "./*")
####

In [ ]:
# Thu thập contents
contents = []

for idx, child in enumerate(children):
    text = child.text.strip()
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
        #### CODE HERE
        contents.append(text)
        ####

    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        #### CODE HERE
         if len(text):
            contents.append(f"[{text}]" if len(text) <= 100 else text)
        ####

    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [ ]:
contents

['Cao tốc Bến Lức - Long Thành đóng vai trò quan trọng trong việc kết nối các tỉnh thành khu vực phía Nam, góp phần thúc đẩy giao thương và phát triển kinh tế trong khu vực. Tuyến đường này sở hữu cảnh quan thiên nhiên hùng vĩ, đặc biệt là đoạn đi qua khu vực rừng ngập mặn.',
 'Dự án tổng chiều dài hơn 57km, cao tốc Bến Lức - Long Thành đi qua địa phận tỉnh Long An, Đồng Nai và TPHCM. Trong đó, đoạn đi qua Đồng Nai dài khoảng 27km, bao gồm 3 gói thầu: A5, A6 và A7.',
 'Được biết, gói thầu A7 được đánh giá là phức tạp nhất của dự án do địa hình nhiều sông ngòi, sình lầy.',
 'Những ngày giữa tháng 7, các công nhân đang gấp rút hoàn thiện những hạng mục cuối cùng tại gói thầu A7 gồm thảm nhựa, lắp đặt tấm chống chói, kẻ vạch sơn đường, biển báo giao thông...',
 'Ông Nguyễn Thiện Đạt, Giám đốc điều hành gói thầu này cho biết, cao tốc Bến Lức - Long Thành có khoảng 3,5km đi qua khu rừng ngập mặn thuộc 2 huyện Long Thành và Nhơn Trạch của tỉnh Đồng Nai.',
 '“Hiện nay, tiến độ thi công gói th

In [ ]:
# Thu thập tác giả
is_author_maybe_last = False
try:
    #### CODE HERE
    author = driver.find_element(By.CSS_SELECTOR, ".article-detail-author__info .name a").text
    ####
except NoSuchElementException:
    is_author_maybe_last = True

# Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
if is_author_maybe_last:
    if len(contents) and len(contents[-1]) < 30:
        author = contents.pop()
    else:
        author = "Uknown"
author

'Hoàng Anh- ĐN'

In [ ]:
driver.close()

#### Chạy thật

In [29]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    #### CODE HERE
    title = driver.find_element(By.CLASS_NAME, "content-detail-title").text
    ####

    # Thu thập description
    #### CODE HERE
    description = driver.find_element(By.CLASS_NAME, "content-detail-sapo").text
    ####

    # Thu thập thể loại
    #### CODE HERE
    cats_el = driver.find_elements(By.CSS_SELECTOR, ".bread-crumb-detail > ul > li > a[title]")
    main_cat = cats_el[0].text if len(cats_el) > 0 else ""
    sub_cat = cats_el[1].text if len(cats_el) > 1 else ""
    ####

    # Thu thập published date
    #### CODE HERE
    publish_date = driver.find_element(By.CLASS_NAME, "bread-crumb-detail__time").text
    ####

    # Thu thập content bài báo
    #### CODE HERE
    article = driver.find_element(By.CLASS_NAME, "main-content")
    children = article.find_elements(By.XPATH, "./*")
    contents = []
    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]:
            contents.append(text)
        elif child.tag_name == "figure" :
            if len(text): contents.append(f"[{text}]" if len(text) <= 100 else text)

    # Thu thập tác giả
    #### CODE HERE
    is_author_maybe_last = False
    try:
        #### CODE HERE
        author = driver.find_element(By.CSS_SELECTOR, ".article-detail-author__info .name a").text
        ####
    except NoSuchElementException:
        is_author_maybe_last = True

    # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
    if is_author_maybe_last:
        if len(contents) and len(contents[-1]) < 30:
            author = contents.pop()
        else:
            author = "Uknown"
    author
    ####


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [31]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    if cat == "Chính trị": continue
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue


    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

# Bug will be emagazines and videos, we ignore them as well

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Thu thập dữ liệu thể loại Chính trị ..
Thu thập dữ liệu thể loại Thời sự ..
Bug at url: https://vietnamnet.vn/ban-hoc-cu-tiec-thuong-tong-bi-thu-nguyen-phu-trong-2305537.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/song-bui-menh-mong-nuoc-bo-doi-chay-dua-giup-dan-ha-noi-chong-lu-2305330.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/nguoi-dan-khoc-nghen-tai-le-thuong-co-ru-quoc-tang-tong-bi-thu-nguyen-phu-trong-2305383.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/truc-thang-dua-hai-benh-nhan-nang-tu-truong-sa-ve-dat-lien-dieu-tri-2307444.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/song-thap-thom-trong-man-dem-den-kit-o-ron-lu-ha-noi-2306781.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/sau-va-cham-voi-xe-may-cung-chieu-nguoi-giao-hang-bi-xe-container-can-tu-vong-2307669.html, with NoSuchElementException
Bug at url: https://vietnamnet.vn/linh-xa-dua-tong-bi-thu-nguyen-phu-tro

In [32]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://vietnamnet.vn/nghe-an-day-manh-dau-tu-ha-tang-giao-thong-trong-xay-dung-nong-thon-moi-2294946.html',
 'title': 'Nghệ An đẩy mạnh đầu tư hạ tầng giao thông trong xây dựng nông thôn mới',
 'description': 'Xác định giao thông là một trong những bước đột phá để phát triển kinh tế - xã hội, bảo đảm an ninh - quốc phòng, tỉnh Nghệ An đã tập trung nhiều nguồn lực đầu tư vào lĩnh vực này.',
 'content': 'Thời gian qua, tỉnh Nghệ An đã ban hành các cơ chế, chính sách gắn việc phát triển hạ tầng giao thông nông thôn với Chương trình mục tiêu quốc gia xây dựng nông thôn mới để từng bước hoàn thiện kết cấu hạ tầng giao thông đường bộ địa phương, kết nối rộng khắp giữa quốc lộ - đường tỉnh – đường xuyên xã – đường thôn, bản, đặc biệt các xã khó khăn.\nTheo đó, nhiều tuyến đường quan trọng, kết nối vùng đã được triển khai đầu tư xây dựng, hoàn thành đưa vào khai thác sử dụng như: Đường nối Quốc lộ 1 đi thị xã Thái Hòa (nay là Quốc lộ 48D), dài 29,123km. Đường giao thông nối đường N5 

: 

## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set to your folder
# FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
# !cp -r data $FOLDER_SAVED_GOOGLE_COLAB